# Within-Visit Time Difference
Determine the value of the hyperparameter `cnfg.VISIT_MERGING_TIME_THRESHOLD`, which controls when to split a visit into two separate visits based on the time difference between its underlying fixations.

In [8]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio
from statsmodels.sandbox.stats.stats_dhuard import percentileofscore

import config as cnfg

# pio.renderers.default = "notebook"
pio.renderers.default = "browser"

### Read data

In [2]:
from analysis.pipeline.full_pipeline import read_saved_data

_targets, _actions, _metadata, _idents, fixations, _visits = read_saved_data()

### Calculate the Temporal and Spatial Difference between Subsequent Fixations

In [39]:
for (subj_id, trial_num, eye), data in fixations.groupby(
    ["subject", "trial", "eye"]
):
    data = data.sort_values("start_time")
    time_diff = (data["start_time"] - data["end_time"].shift(1)).fillna(np.inf)  # Fill the first value with infinity
    fixations.loc[data.index, "time_diff"] = time_diff
    spatial_diff = np.sqrt(
        (data["x"].diff() ** 2) + (data["y"].diff() ** 2)
    ).fillna(np.inf)  # Fill the first value with infinity
    fixations.loc[data.index, "spatial_diff"] = spatial_diff

valid_fixations = fixations.loc[
    np.isfinite(fixations["time_diff"]) &
    (fixations["time_diff"] <= 1000) &          # limit to 1000 ms
    np.isfinite(fixations["spatial_diff"]) &
    (fixations["spatial_diff"] <= 5.0)          # limit to 5 DVA
]

### Plot the Relationship

In [21]:
fig = px.scatter(
    valid_fixations,
    x="time_diff", y="spatial_diff", color="subject",
    marginal_x="violin", marginal_y="violin", log_x=False, log_y=False,
    trendline="ols", trendline_scope="trace", trendline_color_override="black",
    trendline_options=dict(log_x=False, log_y=False),
)
fig.show()

In [41]:
percentiles = [0.05, 0.25, 0.5, 0.75, 0.95]

time_diff_summary = (
    pd.concat([
        valid_fixations["time_diff"].describe(percentiles).rename("all"),
        valid_fixations.groupby("subject")["time_diff"].describe(percentiles).T,
    ], axis=1)
).T

print("Time Difference stats:")
display(time_diff_summary)

spatial_diff_summary = (
    pd.concat([
        valid_fixations["spatial_diff"].describe(percentiles).rename("all"),
        valid_fixations.groupby("subject")["spatial_diff"].describe(percentiles).T,
    ], axis=1)
).T

# print("Spatial Difference stats:")
# display(spatial_diff_summary)

Time Difference stats:


,count,mean,std,min,5%,25%,50%,75%,95%,max
all,3507.0,22.157685,22.129873,8.0,8.0,10.00,17.0,26.00,50.00,269.0
1,107.0,20.719626,11.047747,8.0,8.3,15.00,17.0,24.00,42.90,64.0
2,948.0,25.970464,22.361321,8.0,8.0,15.00,19.0,32.00,60.00,269.0
3,96.0,32.708333,47.910205,8.0,8.0,9.00,15.0,20.00,160.50,190.0
12,207.0,25.734300,28.550278,8.0,8.0,14.00,18.0,25.50,77.70,191.0
13,230.0,15.391304,12.870328,8.0,8.0,9.00,13.0,18.00,25.55,115.0
14,115.0,23.817391,35.696966,8.0,8.0,9.00,15.0,20.00,57.30,208.0
15,97.0,18.979381,27.096271,8.0,8.0,8.00,9.0,17.00,48.20,160.0
16,82.0,28.475610,44.293975,8.0,8.0,15.00,17.0,22.75,55.90,237.0
17,373.0,23.635389,14.670981,8.0,8.0,15.00,27.0,28.00,36.00,215.0


##### Percent of fixations that are within 20ms apart from the previous one

In [42]:
from scipy.stats import percentileofscore

quantiles = valid_fixations.groupby("subject")["time_diff"].apply(lambda x: percentileofscore(x, 20)).rename("quantile")
quantiles.loc["all"] = percentileofscore(valid_fixations["time_diff"], 20)

quantiles

subject
1      68.224299
2      53.322785
3      72.395833
12     60.386473
13     81.956522
14     74.782609
15     81.958763
16     70.731707
17     39.142091
18     81.313131
19     69.148936
20     70.813397
21     70.629371
22     78.027682
all    63.886513
Name: quantile, dtype: float64